In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import alpha_vantage as av
from alpha_vantage.alphavantage import AlphaVantage
from alpha_vantage.techindicators import TechIndicators as ti
from alpha_vantage.alphavantage import csv
from alpha_vantage.timeseries import TimeSeries as ts
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
# Load .env enviroment variables
load_dotenv()

True

In [3]:
# Set email variables
sender_password = os.getenv("SENDER_PASSWORD")
sender_address = 'boltleyfool@gmail.com' 
receiver_address = 'boltleyfool@gmail.com'

### Create Alpaca API object, get account, list positions, and check if market is open

In [4]:
# Specify paper trading environment & set API keys
alpaca_paper_endpoint = os.getenv("ALPACA_PAPER_ENDPOINT")
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
# Create alpaca API object
alpaca_api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    alpaca_paper_endpoint,
    api_version = 'v2'
)
# Get Alpaca Account
account = alpaca_api.get_account()
# Get positions and check market open/closed
clock = alpaca_api.get_clock()
positions = alpaca_api.list_positions()
# Put positions and extra info into DataFrame
positions_df = pd.DataFrame({
    'ticker': [x.symbol for x in positions],
    'qty': [x.qty for x in positions],
    'lastday_price': [x.lastday_price for x in positions],
    'current_price': [x.current_price for x in positions],
    'change_today': [x.change_today for x in positions],
    'market_value': [x.market_value for x in positions],
    'unrealized_intraday_pl': [x.unrealized_intraday_pl for x in positions],
    'unrealized_intraday_plpc': [x.unrealized_intraday_plpc for x in positions],
})
# Display some portfolio data
positions_df.head()

,ticker,qty,lastday_price,current_price,change_today,market_value,unrealized_intraday_pl,unrealized_intraday_plpc
0,RCL,100,67.99,68.5,0.0075011031033976,6850,51,0.0075011031033976
1,NVDA,10,610.03,612.7,0.0043768339262004,6127,26.7,0.0043768339262004
2,GOOG,10,2095.89,2092,-0.0018560134358196,20920,-38.9,-0.0018560134358196
3,FVRR,100,302.91,304,0.0035984285761447,30400,109,0.0035984285761447
4,FTNT,50,163.01,163.01,0,8150.5,0,0


### Define AlphaVantage URL, API key, and variables
* Sample URL for querying a technical indicator on a position:
 * https://www.alphavantage.co/query?function=HT_TRENDLINE&symbol=IBM&interval=daily&series_type=close&apikey=demo

In [5]:
# Get AlphaVantage API Key
av_api_key = os.getenv("ALPHAVANTAGE_API_KEY")
# Set AlphaVantage API URL
av_api = 'https://www.alphavantage.co/query?'
# Define variables used across all technical indicators
interval = '30min'

In [6]:
#Trading algorithm
def get_obv(symbol_obv):
    if clock.is_open==True:
        mail_content="Market is open"
        if bool(positions) == False:
            for obv in symbol_obv:
                query = {'function':'OBV','symbol':p.ticker,'interval':interval,'apikey':av_api_key}
                response = requests.get(av_api, params = query)
                response_dict = response.json()
                _, header = response.json()
            # Convert to DataFrame
                obv_df = pd.DataFrame.from_dict(response_dict[header], orient = 'index')
            # Clean up column names
                df_cols = [i.split(' ')[1] for i in df.columns]
                df.columns = df_cols
                df.set_index('timestamp', inplace=True)
            #ex. OBV rises etc.
            if []:
                api.submit_order(symbol = stockDEF, qty = number_of_shares,side = 'buy',type = 'market' ,time_in_force = 'day')
                mail_content = "Trades have been made, XYZ..."
            #ex. OBV drops etc.
            elif []:
                api.submit_order(stockABC = stockDEF, qty = number_of_shares,side = 'sell',type = 'market', time_in_force ='day')
                mail_content = "Trades have been made, XYZ..." 
        else:
            if bool(positions) == True:
                api.close_position(stockABC)
                api.open_position(stockDEF)
                mail_content = "XYZ positions have been traded"
            else:
                mail_content = "No trades were made, positions remains the same"
                pass
    else:
        mail_content="Market is closed"
# Configure MIME objects
    message = MIMEMultipart()
    message['From'] = 'Botley'
    message['To'] = receiver_address
    message['Subject'] = 'Trade Notification'
# Set body and attachments for the email
    message.attach(MIMEText(mail_content,'plain'))
# Create SMTP session using gmail server
    session = smtplib.SMTP('smtp.gmail.com', 587)
# Start transport layer security
    session.starttls()
# Login sender
    session.login(sender_address,sender_password)
# Set mail_content to string
    text = message.as_string()
# Send the email
    session.sendmail(sender_address,receiver_address,text)
    session.quit()
    
    done = 'OBV Trade Email Sent'

    return done

In [7]:
bool(positions)

True

In [8]:
clock.is_open

False

In [ ]:
obv_df.head()